In [16]:
import tensorflow as tf
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm  import tqdm
import dataprocessor.processor as data_loader
import dataprocessor.util as data_util
import models.model as model_loader
import models.configure as model_config
import json

'''To resolve Running the Tensorflow 2.0 code gives 
   ValueError: tf.function-decorated function tried to create
   variables on non-first call'. What am I doing wrong?
'''
tf.config.run_functions_eagerly(True)

!!apt-get install abcmidi timidity > /dev/null 2>&1

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
# assert len(tf.config.list_physical_devices('GPU')) > 0

[]

In [17]:
#load data for test
songs = data_loader.load_training_data()
print(songs[2])
# Join our list of song strings into a single string containing all songs
songs_joined = "\n\n".join(songs) 

X:3
T:Belfast
Z: id:dc-hornpipe-3
M:C|
L:1/8
K:D Major
ag|(3faf df AdFA|DFAd f2ef|gbec dfAF|GABG E2ag|!
(3faf df AdFA|DFAd f2ef|gbed cABc|d2f2 d2:|!
(3DEF|GFGA Bcde|fgfe dcdB|A2f2 fef2|G2e2 ede2|!
GFGA Bcde|fgfe dcdB|Afed cABc|d2f2 d2:|!
ag|(3fgf (3efe (3ded (3cdc|(3BcB (3ABA G2ba|(3gag (3fgf (3efe (3ded|(3cdc (3BcB A2ag|!
(3fgf (3efe (3ded (3cdc|(3BcB (3ABA (3GAG (3FGF|Eged cABc|d2f2 d2:|!


In [18]:
#load char2index
f = open('char2idx.json')
char2idx = json.load(f)
f.close()
# print(type(char2idx),char2idx.keys())

In [19]:
# Create a mapping from indices to characters. This is
#   the inverse of char2idx and allows us to convert back
#   from unique index to the character in our vocabulary.
idx2char = np.array([iter[0] for iter in list(char2idx.items())])

In [20]:
len(idx2char)

83

In [21]:

idx2char

array(['\n', ' ', '!', '"', '#', "'", '(', ')', ',', '-', '.', '/', '0',
       '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|'], dtype='<U1')

In [22]:
vectorized_songs = data_loader.vectorize_string(songs_joined,char2idx)

In [23]:
print(len(vectorized_songs),len(set(vectorized_songs)))

200679 83


In [24]:
# Build a simple model with default hyperparameters. You will get the 
#   chance to change these later.
vocab_size=len(char2idx)
batch_size=1
model = model_loader.build_model(vocab_size, model_config.embedding_dim, model_config.rnn_units, batch_size)


In [25]:
# Restore the model weights for the last checkpoint after training
model.load_weights(tf.train.latest_checkpoint(model_config.checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            21248     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 83)             85075     
Total params: 5,353,299
Trainable params: 5,353,299
Non-trainable params: 0
_________________________________________________________________


In [26]:
'''Use the model and the function defined above to generate ABC format text of length 1000!
    As you may notice, ABC files start with "X" - this may be a good start string.'''
generated_text = model_loader.generate_text(model,char2idx,idx2char, start_string="X", generation_length=1000)

### Play back generated songs ###

generated_songs = data_loader.extract_song_snippet(generated_text)

for i, song in enumerate(generated_songs): 
  # Synthesize the waveform from a song
  waveform = data_loader.play_song(song)
  print(song)
  # If its a valid song (correct syntax), lets play it! 
  if waveform:
    print("Generated song", i)
    ipythondisplay.display(waveform)

100%|██████████| 1000/1000 [00:05<00:00, 195.56it/s]

X:usc"=Sf eaa eaa2|ecgc acgc|ecgc acgc|eedecAc|dfec d2ge|d|cAGB ADFA|!
DEFD EFGB|A=FF BAGF|GEBED|!
EGBG AGED|E2GA Bdgf|edef ed ef AFD|!
B3 BAB|G3 F2G|ABA AGA Bcd:|!
F2|A4 B2|D2 F2 E2|D2 E2 D2|F4 G2|A4 B2|A3 G F2|F2 Ee dc|d2 df ed|^ce e2|c2 dc BAG A2c|cBc dcB|cAF DEFG A2|]!
ce|Adfd Adfd|Adfd dcBc|Acec Acec|Acec dcBc|!
Adfd adfd|Acce agae|faed cA:|!
Bd|eaag a2ga|bgaf gfed|Bdgd egg2 a2ba|gedB GBdg:|!
X:76
T:Dowd's No. 1
Z: id:dc-polka-37
M:2/4
L:1/8
K:D Major
FE|D2 FG|A2 F2|D4:|!
fgf afdfe|fed BdB|!
AFE EFE|DFA BAB|cAG E2|]!
X:45
T:Dillon's Fancy
Z: id:dc-reel-76
M:C
L:1/8
K:D Mixolydian
ag|fded cAGE|cBcd edcd|edBc d3A|d2eg fe:|!
